In [1]:
#Optimization

In [2]:
from pulp import *
import pandas as pd

In [3]:
foodData = pd.read_excel("diet.xls")   #Read in data
constraint_info = foodData[65:68]

In [4]:
constraint_info  #Constraint data will help us later while we write constraints

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,NaN,NaN,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


In [5]:
foodData.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [6]:
foodData.tail()


,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,NaN,NaN,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


In [7]:
# Dietary information is in rows 1 - 64

In [8]:
foodData = foodData[0:64] #Isolate the dietary information for the model

In [9]:
foodData = foodData.values.tolist()

In [10]:
foodData[0]     #Check list structure

['Frozen Broccoli',
 0.16,
 '10 Oz Pkg',
 73.8,
 0.0,
 0.8,
 68.2,
 13.6,
 8.5,
 8.0,
 5867.4,
 160.2,
 159.0,
 2.3]

In [11]:
foodData[0][0]   #Foods are in 0 index

'Frozen Broccoli'

In [12]:
foods = [x[0] for x in foodData]  #Make a list of foods

In [13]:
cost = dict([(x[0], float(x[1])) for x in foodData])
cals = dict([(x[0], float(x[3])) for x in foodData])

cholesterol = dict([(x[0], float(x[4])) for x in foodData])
fat = dict([(x[0], float(x[5])) for x in foodData])

sodium = dict([(x[0], float(x[6])) for x in foodData])
carbs = dict([(x[0], float(x[7])) for x in foodData])

fiber = dict([(x[0], float(x[8])) for x in foodData])
protein = dict([(x[0], float(x[9])) for x in foodData])

Vit_A = dict([(x[0], float(x[10])) for x in foodData])
Vit_C = dict([(x[0], float(x[11])) for x in foodData])

calcium = dict([(x[0], float(x[12])) for x in foodData])
iron = dict([(x[0], float(x[13])) for x in foodData])


In [14]:
# Define our problem

In [15]:
problem = LpProblem("Diet Problem", LpMinimize)

In [16]:
amountVars = LpVariable.dicts("Amounts", foods, 0)

In [17]:
# Code in our objective function

In [18]:
problem += lpSum( [cost[i] * amountVars[i] for i in foods]), "total cost"

In [19]:
problem += lpSum( [cals[i] * amountVars[i] for i in foods ] ) >= 1500
problem += lpSum( [cals[i] * amountVars[i] for i in foods ] ) <= 2500

problem += lpSum( [cholesterol[i] * amountVars[i] for i in foods ] ) >= 30
problem += lpSum( [cholesterol[i] * amountVars[i] for i in foods ] ) <= 240

problem += lpSum( [fat[i] * amountVars[i] for i in foods ] ) >= 20
problem += lpSum( [fat[i] * amountVars[i] for i in foods ] ) <= 70

problem += lpSum( [sodium[i] * amountVars[i] for i in foods ] ) >= 800
problem += lpSum( [sodium[i] * amountVars[i] for i in foods ] ) <= 2000

problem += lpSum( [carbs[i] * amountVars[i] for i in foods ] ) >= 130
problem += lpSum( [carbs[i] * amountVars[i] for i in foods ] ) <= 450

problem += lpSum( [fiber[i] * amountVars[i] for i in foods ] ) >= 125
problem += lpSum( [fiber[i] * amountVars[i] for i in foods ] ) <= 250

problem += lpSum( [protein[i] * amountVars[i] for i in foods ] ) >= 60
problem += lpSum( [protein[i] * amountVars[i] for i in foods ] ) <= 100

problem += lpSum( [Vit_A[i] * amountVars[i] for i in foods ] ) >= 1000
problem += lpSum( [Vit_A[i] * amountVars[i] for i in foods ] ) <= 10000

problem += lpSum( [Vit_C[i] * amountVars[i] for i in foods ] ) >= 400
problem += lpSum( [Vit_C[i] * amountVars[i] for i in foods ] ) <= 5000

problem += lpSum( [calcium[i] * amountVars[i] for i in foods ] ) >= 700
problem += lpSum( [calcium[i] * amountVars[i] for i in foods ] ) <= 1500

problem += lpSum( [iron[i] * amountVars[i] for i in foods ] ) >= 10
problem += lpSum( [iron[i] * amountVars[i] for i in foods ] ) <= 40



In [20]:
#Create binary variable for whether food is present and dict 
#with lower bounds

In [21]:
# create dictionary of food variables w/zero lower bound
food_vars = LpVariable.dicts('choices', foods, lowBound=0)

# create binary variable to indicate whether food is included
food_present = LpVariable.dicts('included', foods, lowBound=0, upBound=1, cat=LpBinary)

In [22]:
#At least 1/10 if food is preset

In [23]:
for food in foods:
    problem += food_present[food] * 100000 >= amountVars[food]
    problem += amountVars[food] >= food_present[food] * 0.1

In [24]:
#Only broccoli or celery should be present

In [25]:
problem += lpSum(food_present['Frozen Broccoli'] + 
                 food_present['Celery, Raw']) <= 1, 'Celery or Broccoli'

In [26]:
#Create list of protein foods

In [27]:
protein_foods = ['Roasted Chicken', 'Poached Eggs', 'Scrambled Eggs', 
                 'Bologna,Turkey', 'Frankfurter, Beef', 
                 'Ham,Sliced,Extralean', 'Kielbasa,Prk', 
                 'Hamburger W/Toppings', 'Hotdog, Plain', 'Pork', 
                 'Sardines in Oil', 'White Tuna in Water']

In [28]:
#Get at least 3 high-protein foods

In [29]:
problem += lpSum(food_present[i] for i in protein_foods) >= 3

In [30]:
problem.solve()

1

In [31]:
print ("Status:" ,LpStatus[problem.status])

Status: Optimal


In [32]:
for v in problem.variables():
    print(v.name, v.varValue)

Amounts_2%_Lowfat_Milk 0.0
Amounts_3.3%_Fat,Whole_Milk 0.0
Amounts_Apple,Raw,W_Skin 0.0
Amounts_Apple_Pie 0.0
Amounts_Bagels 0.0
Amounts_Banana 0.0
Amounts_Beanbacn_Soup,W_Watr 0.0
Amounts_Bologna,Turkey 0.0
Amounts_Butter,Regular 0.0
Amounts_Cap'N_Crunch 0.0
Amounts_Carrots,Raw 0.0
Amounts_Celery,_Raw 42.399358
Amounts_Cheddar_Cheese 0.0
Amounts_Cheerios 0.0
Amounts_Chicknoodl_Soup 0.0
Amounts_Chocolate_Chip_Cookies 0.0
Amounts_Corn_Flks,_Kellogg'S 0.0
Amounts_Couscous 0.0
Amounts_Crm_Mshrm_Soup,W_Mlk 0.0
Amounts_Frankfurter,_Beef 0.0
Amounts_Frozen_Broccoli 0.0
Amounts_Frozen_Corn 0.0
Amounts_Grapes 0.0
Amounts_Ham,Sliced,Extralean 0.0
Amounts_Hamburger_W_Toppings 0.0
Amounts_Hotdog,_Plain 0.0
Amounts_Kielbasa,Prk 0.1
Amounts_Kiwifruit,Raw,Fresh 0.0
Amounts_Lettuce,Iceberg,Raw 82.802586
Amounts_Macaroni,Ckd 0.0
Amounts_Malt_O_Meal,Choc 0.0
Amounts_New_E_Clamchwd,W_Mlk 0.0
Amounts_Neweng_Clamchwd 0.0
Amounts_Oatmeal 0.0
Amounts_Oatmeal_Cookies 0.0
Amounts_Oranges 3.0771841
Amounts_Pea